## Scenario 1: A single data scientist participating in an ML competition

This scenario demonstrates how an individual data scientist can use MLflow to track machine learning experiments on their local machine. This is a common setup for solo projects, hackathons, or competitions where collaboration and remote access are not required.

### MLflow setup overview
- **Tracking server:** Not used (runs locally, no remote server)
- **Backend store:** Local filesystem (stores experiment metadata in the `mlruns/` folder)
- **Artifacts store:** Local filesystem (stores model files and other artifacts in the same `mlruns/` folder)

With this setup, all experiment runs, parameters, metrics, and artifacts are saved locally. You can explore and compare your experiments using the MLflow UI.

### How to use the MLflow UI
- You can launch the MLflow UI by running `mlflow ui` in your terminal, or by running the provided code cell in this notebook.
- The UI will be available at [http://localhost:5000](http://localhost:5000).
- Use the UI to browse experiments, compare runs, and inspect logged models and artifacts.

> **Tip:** This local setup is ideal for learning and prototyping. For team projects or production, you would typically use a remote tracking server and a more robust backend (e.g., a database and cloud storage).

In [1]:
%pip install mlflow
import mlflow

  Using cached mlflow-3.1.4-py3-none-any.whl.metadata (29 kB)
  Using cached flask-3.1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached gunicorn-23.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached matplotlib-3.9.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached pandas-2.3.3-cp39-cp39-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached scikit_learn-1.6.1-cp39-cp39-macosx_12_0_arm64.whl.metadata (31 kB)
  Using cached cachetools-6.2.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached databricks_sdk-0.70.0-py3-none-any.whl.metadata (39 kB)
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached opentelemetry_api-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached

/Users/meghanavastare/Documents/IE MLOPS/ie-mlops-nyc-taxis/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///Users/meghanavastare/Documents/IE%20MLOPS/ie-mlops-nyc-taxis/experiment_tracking/mlruns'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///Users/meghanavastare/Documents/IE%20MLOPS/ie-mlops-nyc-taxis/experiment_tracking/mlruns/0', creation_time=1761735185032, experiment_id='0', last_update_time=1761735185032, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Note: you may need to restart the kernel to use updated packages.


### Creating an experiment and logging a new run

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import mlflow
import sklearn
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

mlflow.set_experiment("my-experiment-1")

X, y = load_iris(return_X_y=True)
class_names = load_iris().target_names

models = [
    ("LogisticRegression", LogisticRegression(C=1.0, random_state=42, max_iter=1000, solver="lbfgs")),
    ("RandomForestClassifier", RandomForestClassifier(n_estimators=100, random_state=42))
]

for model_name, model in models:
    with mlflow.start_run() as run:
        mlflow.set_tag("model_type", model_name)
        mlflow.log_param("sklearn_version", sklearn.__version__)
        model.fit(X, y)
        y_pred = model.predict(X)
        acc = accuracy_score(y, y_pred)
        mlflow.log_metric("accuracy", acc)
        # Log confusion matrix as labeled DataFrame
        cm = confusion_matrix(y, y_pred)
        cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
        cm_df.to_csv("confusion_matrix_labeled.csv")
        mlflow.log_artifact("confusion_matrix_labeled.csv")
        # Confusion matrix heatmap as image
        plt.figure(figsize=(5,4))
        sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
        plt.title(f"Confusion Matrix ({model_name})")
        plt.ylabel("True label")
        plt.xlabel("Predicted label")
        plt.tight_layout()
        plt.savefig("confusion_matrix_heatmap.png")
        plt.close()
        mlflow.log_artifact("confusion_matrix_heatmap.png")
        # Provide input_example and use 'name' instead of deprecated 'artifact_path'
        input_example = np.expand_dims(X[0], axis=0)
        mlflow.sklearn.log_model(model, name="models", input_example=input_example)
        mlflow.set_tag("n_classes", len(np.unique(y)))
        mlflow.set_tag("run_time", datetime.datetime.now().isoformat())
        mlflow.set_tag("description", f"{model_name} on Iris dataset")
        print(f"Logged run for {model_name}, accuracy={acc:.3f}")

2025/10/29 11:55:20 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


Logged run for LogisticRegression, accuracy=0.973
Logged run for RandomForestClassifier, accuracy=1.000


In [7]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///Users/meghanavastare/Documents/IE%20MLOPS/ie-mlops-nyc-taxis/experiment_tracking/mlruns/357245253981004653', creation_time=1761735320760, experiment_id='357245253981004653', last_update_time=1761735320760, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///Users/meghanavastare/Documents/IE%20MLOPS/ie-mlops-nyc-taxis/experiment_tracking/mlruns/0', creation_time=1761735185032, experiment_id='0', last_update_time=1761735185032, lifecycle_stage='active', name='Default', tags={}>]

In [8]:
# Launch MLflow UI (default: uses local mlruns/ folder)
import subprocess
import sys

# This will run 'mlflow ui' as a background process
subprocess.Popen([sys.executable, '-m', 'mlflow', 'ui'])
print("MLflow UI started. Open http://localhost:5000 in your browser.")

MLflow UI started. Open http://localhost:5000 in your browser.


/Users/meghanavastare/Documents/IE MLOPS/ie-mlops-nyc-taxis/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[2025-10-29 11:55:32 +0100] [90515] [INFO] Starting gunicorn 23.0.0
[2025-10-29 11:55:32 +0100] [90515] [INFO] Listening at: http://127.0.0.1:5000 (90515)
[2025-10-29 11:55:32 +0100] [90515] [INFO] Using worker: sync
[2025-10-29 11:55:32 +0100] [90516] [INFO] Booting worker with pid: 90516
/Users/meghanavastare/Documents/IE MLOPS/ie-mlops-nyc-taxis/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[2025-10-29 11:55:32 +0100] [90517] [INFO] Booting worker with pid: 90517
[2025-10-29 11:55

### Interacting with the model registry

In [15]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [ ]:
from mlflow.exceptions import MlflowException

try:
    client.search_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")

[2025-06-29 10:04:13 +0000] [87272] [INFO] Starting gunicorn 23.0.0
[2025-06-29 10:04:13 +0000] [87272] [INFO] Listening at: http://127.0.0.1:5000 (87272)
[2025-06-29 10:04:13 +0000] [87272] [INFO] Using worker: sync
[2025-06-29 10:04:13 +0000] [87273] [INFO] Booting worker with pid: 87273
[2025-06-29 10:04:13 +0000] [87274] [INFO] Booting worker with pid: 87274
[2025-06-29 10:04:13 +0000] [87281] [INFO] Booting worker with pid: 87281
[2025-06-29 10:04:13 +0000] [87282] [INFO] Booting worker with pid: 87282
